In [ ]:
# @todo description

In [ ]:
import cv2
from enum import Enum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
# Params
figsmall = 5
figbig = 15
imgfile = '/media/docker/workspace/xraybones/res/xray2.png'

## Normalize Images (Size, Histogram, etc)

In [ ]:
imgraw = cv2.imread(imgfile)
imgraw = cv2.cvtColor(src=imgraw, code=cv2.COLOR_BGR2GRAY)

# Convert to grayscale
imgeq = imgraw.copy()

# Crop image
cropping = 0.08
imgeq = imgeq[ int(cropping*imgeq.shape[0]):int((1.-cropping)*imgeq.shape[0]), 
          int(cropping*imgeq.shape[1]):int((1.-cropping)*imgeq.shape[1])]

# Scale to width: 1000
[height, width] = imgeq.shape
scale=1000./width
imgeq = cv2.resize(imgeq, 
                   (int(imgeq.shape[1]*scale), int(imgeq.shape[0]*scale)), 
                   interpolation = cv2.INTER_AREA)

# Spread histogram along whole image
imgeq=cv2.equalizeHist(src=imgeq)

# Visualize results
fig = plt.figure(figsize=(figbig, figsmall));
plt.subplot(131);

hist=cv2.calcHist(imgraw,[0],None,[256],[0,256])
plt.plot(hist)
hist=cv2.calcHist(imgeq,[0],None,[256],[0,256])
plt.plot(hist)
plt.legend(['old image', 'new image'])

plt.subplot(132);
plt.xticks([]),plt.yticks([]); 
plt.imshow(imgraw, cmap='gray')


plt.subplot(133);
plt.xticks([]),plt.yticks([]); 
plt.imshow(imgeq, cmap='gray')

## Hand and Finger Detection

In [ ]:
# Extract convex hull and defects
# https://medium.com/analytics-vidhya/hand-detection-and-finger-counting-using-opencv-python-5b594704eb08
# https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html
fig = plt.figure(figsize=(figbig, figbig))

# Loading exposure images into a list
img = imgeq.copy()

# Blur image
img_blur = cv2.GaussianBlur(img,(15,15),0)

# Threshold (binary image)
low_thres = np.average(imgeq)
ret,img_thres = cv2.threshold(img_blur, low_thres, 255, cv2.THRESH_BINARY)
img_eq = cv2.equalizeHist(img_thres)

plt.subplot(1,3,1); plt.xticks([]),plt.yticks([]); plt.imshow(img_eq, cmap='gray')

# Morphological (remove outliers)
kernel = np.ones((15, 15), 'uint8')
img_morph = cv2.morphologyEx(img_eq, cv2.MORPH_CLOSE, kernel)

plt.subplot(1,3,2); plt.xticks([]),plt.yticks([]); plt.imshow(img_morph, cmap='gray')

# Contours
contours, hierarchy = cv2.findContours(img_morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = max(contours, key=lambda x: cv2.contourArea(x))

# Convex hull
hull = cv2.convexHull(contours)

# Hull defects
hull = cv2.convexHull(contours, returnPoints=False)
defects = cv2.convexityDefects(contours, hull)

img2 = imgeq.copy()
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

plt.subplot(1,3,3); plt.xticks([]),plt.yticks([]); plt.imshow(img2, cmap='gray')

In [ ]:
img2 = imgeq.copy()

if defects is not None:
    defcnt = 0

fingergaps = np.array([])

for i in range(defects.shape[0]):  # calculate the angle
    s, e, f, d = defects[i][0]
    start = tuple(contours[s][0])
    end = tuple(contours[e][0])
    far = tuple(contours[f][0])
    a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
    b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
    c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  #      cosine theorem
    if angle <= np.pi / 2:  # angle less than 90 degree, treat as fingers
        defcnt += 1
        cv2.circle(img2, far, 10, [255, 0, 0], -1)
        fingergaps = np.append(fingergaps, [far[0], -far[1]])
        
fingergaps = fingergaps.flatten().reshape(-1,2)

if defcnt > 0:
    defcnt = defcnt+1
    
# Visualize
fig = plt.figure(figsize=(figbig, figbig))

cv2.putText(img2, '%d Fingers detected' % defcnt, (0, 50), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2, cv2.LINE_AA)
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

In [ ]:
# Visualize convex hull and defects
img2 = imgeq.copy()

cnt = contours

defects_arr = np.array([])

for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = tuple(cnt[s][0])
    end = tuple(cnt[e][0])
    far = tuple(cnt[f][0])
    
    if d > 500:
        continue
    elif far[1] > 0.9*img2.shape[0]: # ignore points at bottom
        continue
    defects_arr = np.append(defects_arr, [far[0], -far[1]])
    #cv.circle(img2,far,10,[255,0,0],-1)

# K-Means Clustering of fingertip defects
defects_arr = defects_arr.reshape(-1,2)
kmeans = KMeans(n_clusters=5, random_state=0).fit(defects_arr)
kmeans.labels_
fingertips = kmeans.cluster_centers_

In [ ]:
# Crop Hand contour
imghand = imgeq.copy()

mask = np.zeros_like(imghand) # Create mask where white is what we want, black otherwise
cv2.drawContours(image=mask, contours=[contours], 
                 contourIdx=0, color=(255, 255, 255), thickness=-1) # Draw filled contour in mask
imghand = np.zeros_like(imghand) # Extract out the object and place into output image
imghand[mask == 255] = img[mask == 255]

# Show the output image
fig = plt.figure(figsize=(figsmall,figsmall))
plt.imshow(imghand, cmap='gray')

In [ ]:
# Visualize
fig = plt.figure(figsize=(figbig, figbig))

cv2.putText(img2, '%d Fingers detected' % defcnt, (0, 50), 
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2, cv2.LINE_AA)
cv2.drawContours(img2, [contours], -1, (255, 255, 255), 2)

fig = plt.figure(figsize=(figbig, figbig))
plt.imshow(imghand, cmap='gray')

for fingergap in fingergaps:
    plt.scatter(fingergap[0], -fingergap[1], c='r', marker='.', s=200)
    
for fingertip in fingertips:
    plt.scatter(fingertip[0], -fingertip[1], c='r', marker='x', s=200)

## Image Convolution With Existing Image

## Bone Detection (Canny Edge)

In [ ]:
# Canny Edge Detector
img = imghand.copy()
img_blur = cv2.blur(img, (6,6))
img_canny = cv2.Canny(img_blur, 0, 30, 100)

# draw hand contour overlay
cv2.drawContours(image=img_canny, contours=[contours], 
                 contourIdx=0, color=(0,0,0), thickness=5)

fig = plt.figure(figsize=(figbig,figbig))
plt.imshow(img_canny, cmap='gray')

In [ ]:
# Visualize image
image_raw_r = cv2.cvtColor(src=img_canny, code=cv2.COLOR_GRAY2RGB)*100
image_raw_r[:, :, 0] = 0
image_raw_r[:, :, 2] = 0

img = imghand.copy()
img = cv2.cvtColor(src=img, code=cv2.COLOR_GRAY2RGB)


# Visualize data
fig = plt.figure(figsize=(figbig,figbig))
vis = cv2.addWeighted(img, 0.3, image_raw_r, 1.0,0)

for fingergap in fingergaps:
    plt.scatter(fingergap[0], -fingergap[1], c='r', marker='.', s=200)
    
for fingertip in fingertips:
    plt.scatter(fingertip[0], -fingertip[1], c='r', marker='x', s=200)

plt.imshow(vis, cmap='gray')

# Legacy Code

In [ ]:
cv2.Canny?

## Apply HDR
https://docs.opencv.org/3.4/d2/df0/tutorial_py_hdr.html

## Detect Shapes
https://www.pyimagesearch.com/2016/02/08/opencv-shape-detection/

## K-Means Clustering
https://realpython.com/k-means-clustering-python/
https://scikit-learn.org/stable/modules/clustering.html#clustering
https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html#sphx-glr-auto-examples-cluster-plot-dbscan-py

## Image Filtering
https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html

## Convolution
https://medium.com/analytics-vidhya/2d-convolution-using-python-numpy-43442ff5f381

## Use k-means to remove background color intensity

## Image Operations

Links:
https://heartbeat.fritz.ai/opencv-python-cheat-sheet-from-importing-images-to-face-detection-52919da36433
https://www.kdnuggets.com/2019/08/introduction-image-segmentation-k-means-clustering.html
https://docs.opencv.org/3.4/d8/dbc/tutorial_histogram_calculation.html